# Clustering with GPT-3.5-turbo, GPT-4, and Langchain

In [19]:
import os
import tiktoken
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from dotenv import load_dotenv
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [20]:
from dotenv import load_dotenv
load_dotenv()
import openai

### Dataset is 1,200 DS related job posting from Indeed

In [3]:
# df = pd.read_csv('./data/data_science_jobs_indeed_usa.csv')

In [4]:
# encoding = tiktoken.get_encoding('cl100k_base')

In [5]:
# df = df.assign(token_count=df['Description'].apply(lambda x: len(encoding.encode(x))))

In [6]:
# print(df.shape)
# df.head()

(1200, 11)


,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,NaN,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25
2,2,IT Business Intelligence Developer (FT) Remote...,Ballad Health,"Remote in Blountville, TN",3.0,PostedPosted 30+ days ago,NaN,Job Details Apply Save Print this job Email a…,https://www.indeed.com/rc/clk?jk=58612836c63b8...,Job Details\nApply\nSave\nPrint this job\nEmai...,10
3,3,Data Engineer,Longevity Holdings Inc.,"Remote in Minneapolis-Saint Paul, MN",NaN,PostedPosted 3 days ago,"$90,000 - $110,000 a year",Incorporate core data management competencies ...,https://www.indeed.com/company/TwentyFirst/job...,Position: Data Engineer\nLocation: MN\nAs a Da...,29
4,4,Network Administrator/dba developer,WKI Kenworth,"Wichita, KS 67219",NaN,EmployerActive 2 days ago,"$50,000 - $70,000 a year",The Network Administrator provides 2nd level e...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Full Job Description\nThe Network Administrato...,28


In [3]:
# Azure: 
# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
# def get_embedding(text) -> list[float]:
#     text = text.replace("\n", " ")
#     return openai.Embedding.create(input=text, engine=OpenAiEmbedding)["data"][0]["embedding"]

# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
# def get_embedding(text, model="text-embedding-ada-002"):
#    text = text.replace("\n", " ")
#    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [8]:
# df = df.assign(embedding=df['Description'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002')))

In [9]:
# df.to_parquet("job_descriptions_with_embeddings.parquet")

In [21]:
df = pd.read_parquet("job_descriptions_with_embeddings.parquet")

In [22]:
df.shape

(1200, 12)

In [23]:
df.head()

,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ..."
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25,"[-0.012073525227606297, -0.026480479165911674,..."
2,2,IT Business Intelligence Developer (FT) Remote...,Ballad Health,"Remote in Blountville, TN",3.0,PostedPosted 30+ days ago,None,Job Details Apply Save Print this job Email a…,https://www.indeed.com/rc/clk?jk=58612836c63b8...,Job Details\nApply\nSave\nPrint this job\nEmai...,10,"[-0.021908748894929886, -0.002960818586871028,..."
3,3,Data Engineer,Longevity Holdings Inc.,"Remote in Minneapolis-Saint Paul, MN",NaN,PostedPosted 3 days ago,"$90,000 - $110,000 a year",Incorporate core data management competencies ...,https://www.indeed.com/company/TwentyFirst/job...,Position: Data Engineer\nLocation: MN\nAs a Da...,29,"[-0.017482835799455643, -0.01076465379446745, ..."
4,4,Network Administrator/dba developer,WKI Kenworth,"Wichita, KS 67219",NaN,EmployerActive 2 days ago,"$50,000 - $70,000 a year",The Network Administrator provides 2nd level e...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Full Job Description\nThe Network Administrato...,28,"[0.0011034636991098523, -0.000915585900656879,..."


In [24]:
X = np.vstack(df.embedding.values)

* `n_clusters=6` is arbitrary & just for example

In [26]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=42)
df['kmeans_label'] = kmeans.fit_predict(X)

In [60]:
print(df.shape)
df.head(2)

(1200, 14)


,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding,kmeans_label,topic_title
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ...",4,"""Driving Data Science and Analytics: Insights,..."
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25,"[-0.012073525227606297, -0.026480479165911674,...",2,"""Job Opportunities in Cloud Migration, Enginee..."


In [28]:
df.token_count.describe()

count    1200.000000
mean       26.441667
std         4.249507
min         9.000000
25%        24.000000
50%        26.000000
75%        29.000000
max        45.000000
Name: token_count, dtype: float64

In [29]:
from src_index.token_catcher import Usage

usage = Usage()

### This method stuffs in-cluster texts to LLM in sliding window style. 
* Text from each cluster will be broken up into batches of ~4k tokens, each will get it's own topic label
* Similar topics suggest more well defined cluster

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

def get_prompt():
    system_template = "You're an expert journalist. You're helping me write a concise topic title for news articles."
    human_template = "Using the following articles, write a topic title that summarizes them.\n\nARTICLES:{articles}\n\nTOPIC TITLE:"

    return ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(system_template),
            HumanMessagePromptTemplate.from_template(human_template),
        ],
        input_variables=["articles"],
    )

# Maximum number of tokens that can be processed at once
max_tokens = 4096

# Average number of tokens per paragraph
avg_tokens_per_paragraph = 50

# Maximum number of paragraphs that can be processed at once
max_paragraphs = max_tokens // avg_tokens_per_paragraph

for c in df.kmeans_label.unique():
    chain = LLMChain(
        llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"), prompt=get_prompt(), verbose=False
    )
    articles = df.query(f"kmeans_label == {c}").to_dict(orient="records")

    
    # Process the articles in chunks
    results = []
    for i in range(0, len(articles), max_paragraphs):
        chunk = articles[i:i+max_paragraphs]
        articles_str = "\n".join([article['Description'] for article in chunk])
        result = chain.run(
            {
                "articles": articles_str,
            }
        )
        results.append(result)
        # Assign the generated topic title to each article in the chunk
        for article in chunk:
            df.loc[df.Description == article['Description'], "topic_title"] = result


In [34]:
usage.total_tokens()

69269

In [35]:
df['topic_title'].value_counts()

topic_title
"Job Requirements and Skills for Data Scientists and Data Engineers in Various Industries"                                           94
"Driving Data Science Initiatives and Building Data-Driven Solutions: A Comprehensive Overview"                                      89
"Business Analysts and Business Intelligence Developers: Roles, Responsibilities, and Skills"                                        85
"Building and Managing Data Analytics Platforms: A Comprehensive Guide"                                                              84
"Machine Learning Engineer: Developing, Deploying, and Testing ML Models for Various Applications"                                   82
"Data Science and Analytics: Experience, Skills, and Roles in the Industry"                                                          74
"Database Administrator/Developer Needed to Resolve Performance Issues and Enhance Database Efficiency"                              74
"Machine Learning Experts and Engine

#### Topics by Cluster Label

In [37]:
df.groupby('kmeans_label')['topic_title'].value_counts()

kmeans_label  topic_title                                                                                                                      
0             "Database Administrator/Developer Needed to Resolve Performance Issues and Enhance Database Efficiency"                              74
              "Database Administrator: Support, Optimization, and Security for Various Database Systems"                                           72
1             "Machine Learning Engineer: Developing, Deploying, and Testing ML Models for Various Applications"                                   82
              "Machine Learning Experts and Engineers: Proficiency, Experience, and Applications"                                                  73
              "Job Opportunities for Experienced Machine Learning Professionals"                                                                   25
2             "Job Opportunities in Cloud Migration, Engineering, and Data Science"                       

In [62]:
# sort clusters by their size, and create a new dataframe with the size and cluster number in sorted order. Maintain other columns
size_and_cluster_number = pd.DataFrame(df.groupby("kmeans_label").size().sort_values(ascending=False))

# print out 10 messages from n cluster
for cluster_number, size in size_and_cluster_number.iterrows():
    print(f"{cluster_number} has {size} messages")
    print(df[df.kmeans_label == cluster_number].sample(4).Description.values)
    print("\n")

4 has 0    281
Name: 4, dtype: int64 messages
['You will be responsible for developing data and machine learning pipelines that handle data across domains including marketing, supply chain, forecasting,…'
 'Help architect data solutions/frameworks and define data models for the underlying data warehouse and data marts.\nDevelop UAT plans and conduct QA.'
 'Provide leadership and technical to other software developers.\nYou will be a member of the BI and Data Science team, you will help establish software for tasks…'
 'Collaboratively review design, code, test plans and dataset implementation performed by other data engineers in support of maintaining data engineering…']


3 has 0    274
Name: 3, dtype: int64 messages
['Master’s degree (or its equivalent) in statistics, mathematics, economics, computer science, data sciences, predictive modeling, or other quantitative…'
 'Perform exploratory data analysis and statistical modelling with high quality data handling techniques.\nAbility to 

In [55]:
# print out 10 messages from n cluster
for i  in range(len(counts_df)):
    print(f"Cluster {counts_df.iloc[i]['kmeans_label']} has {counts_df.iloc[i][0]} cases")
    print("Some random examples:")
    print(df[df.kmeans_label == label].sample(4).Description.values)
    print("\n")

Cluster 4 has 281 cases
Some random examples:
['It takes a scientist to analyze, evaluate, and maintain data so that it can be used to build, validate, and deploy _machine learning (ML) models_ that help SAIF…'
 "This role will be responsible for developing computer vision-based machine learning algorithms and analytic pipelines to drive Caris' research initiatives…"
 'Develop custom data models and algorithms to apply to data sets.\nMachine Learning – good knowledge of machine learning methods like k-Nearest Neighbors, Naive…'
 'As a part of the larger Data Science & Engineering department, the User Experiences ML & CI team produces ML & CI models used to understand the…']


Cluster 3 has 274 cases
Some random examples:
['2+ years of experience as a machine learning engineer or data scientist.\nExperience with machine learning tools, including TensorFlow, Pytorch, or Scikit Learn.'
 'Experience in machine learning (ML) using TensorFlow, TensorRT and CUDA, OpenCL on GPUs is a plus.\nAb

In [63]:
df.groupby('kmeans_label').agg({
    'token_count': ['sum', 'mean']
})

token_count           
                     sum       mean
kmeans_label                       
0                   3641  24.938356
1                   4766  26.477778
2                   3109  28.787037
3                   7529  27.478102
4                   7419  26.402135
5                   5266  24.957346

### Sample questions
We can sample around 4k tokens worth of questions. To do this, we shuffle the questions in each cluster, then cut the first ~4k token chunk from each cluster. This should give us ~50% sample ratio on average

In [69]:
from langchain.text_splitter import CharacterTextSplitter

def create_doc(messages, max_tokens=3800):
    #sample around 125 messages from the cluster
    input_doc = '\n\n'.join(messages)

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=max_tokens, chunk_overlap=0, separator="\n\n")
    # Sanity check
    split_texts = text_splitter.split_text(input_doc)
    
    # text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=16000,chunk_overlap=0,separator="\n\n")
    return split_texts[0]


# for clusters, create docs and save in list
docs = []
for cluster_number, size in size_and_cluster_number.iterrows():
    print(f"{cluster_number} has {size} messages")
    Description = df[df.kmeans_label == cluster_number].Description.values
    doc = create_doc(Description)
    print(doc)
    print("\n\n")
    docs.append(doc)

4 has 0    281
Name: 4, dtype: int64 messages
You’ll be working alongside a team of eight analysts & data scientists - collaborating to design datasets, develop analytical models & provide insights.

Incorporate core data management competencies including data governance, data security, and data quality.
Partner closely with our data scientists to ensure the…

Ensure adaption of data science product with the business users to drive outcome and patient impact.
Ability to work with marketing agencies to define the data…

Stay aware of emerging data science techniques, technologies, and applications.
Hire, train, retain and challenge best-in-class data science and analytics…

Define a clear data science and analytics roadmap.
You will define and execute the product analytics and data science strategy that ensures our organization…

Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM…


### Set up Map Reduce

In [100]:
from langchain import PromptTemplate


map_template_string = """The following is a list of texts that have been clustered using K-Means.
{questions}

Based on this list of questions, please do 3 things: 
(1) Identify the main theme
(2) Give a list of 3 to 5 main sub-themes
(3) Give a representitive example text in each sub-theme
(4) Estimate the proportion of questions that fall into each theme
(5) Share observations on outliers, i.e. questions that do not seem to belong

Helpful Answer:"""

MAP_PROMPT = PromptTemplate(input_variables=["questions"], template=map_template_string)

In [101]:
def run_map(input_doc):

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)
   
    return map_llm_chain.run(questions=input_doc)

In [102]:
usage = Usage()

* Single cluster test with GPT-3.5-turbo

In [103]:
gpt_35_test_result = run_map(docs[0])

In [104]:
usage.total_tokens()

3880

In [105]:
print(gpt_35_test_result)

Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example text: Define a clear data science and analytics roadmap.
   - Proportion: 10%

Observations on outliers

In [106]:
def run_map_gpt4(input_doc):

    llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)
   
    return map_llm_chain.run(questions=input_doc)

In [107]:
usage = Usage()

* Single  test with GPT-4

In [108]:
gpt_4_test_result = run_map_gpt4(docs[0])

In [109]:
usage.total_tokens()

4008

In [110]:
print(gpt_4_test_result)

(1) Main Theme: The main theme of these texts is Data Science and Analytics roles and responsibilities in various organizations.

(2) Sub-themes:
   a. Data Management and Governance
   b. Data Analysis and Insights
   c. Data Science Strategy and Leadership
   d. Collaboration with Other Teams
   e. Data Science Education and Training

(3) Representative Texts:
   a. Data Management and Governance: "Incorporate core data management competencies including data governance, data security, and data quality. Partner closely with our data scientists to ensure the…"
   b. Data Analysis and Insights: "The Data Scientist will be responsible for modeling complex scientific clinical data, discovering business/clinical/health insights and identifying…"
   c. Data Science Strategy and Leadership: "Define a clear data science and analytics roadmap. You will define and execute the product analytics and data science strategy that ensures our organization…"
   d. Collaboration with Other Teams: "Colla

In [111]:
usage = Usage()

### GPT-4 Map Reduce

In [112]:
gpt4_results = []

for input_doc in docs:
    result=run_map(input_doc)
    gpt4_results.append(result)

In [52]:
usage.total_tokens()

23889

In [114]:
#print results
for i, result in enumerate(gpt4_results):

    #print size of cluster
    print(f"Cluster {i} has {size_and_cluster_number.iloc[i].values[0]} messages: \n")

    #print the summary
    print(result)

    #print separator
    print("\n===============================================================\n\n")

Cluster 0 has 281 messages: 

Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example text: Define a clear data science and analytics roadmap.
   - Proportion:

In [115]:
usage = Usage()

### GPT-3.5-Turbo Map Reduce

In [116]:
gpt35_results = []

for input_doc in docs:
    result=run_map(input_doc)
    gpt35_results.append(result)

In [117]:
usage.total_tokens()

22715

In [118]:
#print results
for i, result in enumerate(gpt35_results):

    #print size of cluster
    print(f"Cluster {i} has {size_and_cluster_number.iloc[i].values[0]} messages: \n")

    #print the summary
    print(result)

    #print separator
    print("\n===============================================================\n\n")

Cluster 0 has 281 messages: 

Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example text: Define a clear data science and analytics roadmap.
   - Proportion:

### Send all cases through Map Reduce chain

In [119]:
from langchain.text_splitter import CharacterTextSplitter
input_doc = '\n\n'.join(docs)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=4000,chunk_overlap=0,separator="\n\n")
# Sanity check
len(text_splitter.split_text(input_doc))

6

In [120]:
map_template_string = """The following is a list of texts that have been clustered using K-Means
{questions}

Based on this list of texts, please do 4 things: 
(1) identify the main themes 
(2) give a represntitive example question in each theme
(3) estimate the proportion of questions that fall into each theme
(4) share observations on outliers, i.e. questions that do not seem to belong

Helpful Answer:"""

reduce_template_string = template = """The following is a list of summaries for questions entered into a Q+A system:
{question_summaries}

Take these and distill it into a final, consolidated list with: 
(1) the main question themes 
(2) two represntitive example questions in each theme
(3) estimate the proportion of questions that fall into each theme
(4) a summary of outliers

Helpful Answer:"""
MAP_PROMPT = PromptTemplate(input_variables=["questions"], template=map_template_string)
REDUCE_PROMPT = PromptTemplate(input_variables=["question_summaries"], template=reduce_template_string)

In [139]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import (
                StuffDocumentsChain,
                LLMChain,
                ReduceDocumentsChain,
                MapReduceDocumentsChain,
            )


def run_map_reduce(input_doc, model, MAP_PROMPT, REDUCE_PROMPT):
    
    llm = ChatOpenAI(model_name=model, temperature=0)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)

    llm = ChatOpenAI(model_name=model, temperature=0)
    reduce_llm_chain = LLMChain(llm=llm, prompt=REDUCE_PROMPT)

    # Takes a list of documents and combines them into a single string
    combine_documents_chain = StuffDocumentsChain(
            llm_chain=reduce_llm_chain,
            document_variable_name="question_summaries")
    
    # Combines and iteravely reduces the mapped documents 
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `combine_documents_chain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=4000)

    # Combining documents by mapping a chain over them, then combining results
    combine_documents = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_llm_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="questions",
        # Return the results of the map steps in the output
        ### Bug: this currently does not work ###
        return_intermediate_steps=False)
        
    # Define Map=Reduce
    map_reduce = MapReduceChain(
        # Chain to combine documents
        combine_documents_chain=combine_documents,
        # Splitter to use for initial split
        text_splitter=text_splitter)
    
    return map_reduce.run(input_text=input_doc)

In [123]:
len(input_doc)

117588

In [125]:
len(docs[0])

20346

In [133]:
usage = Usage()

In [134]:
map_reduce_gpt4_res = []
for i in range(len(docs)):
    result = run_map_reduce(docs[i], "gpt-4",MAP_PROMPT, REDUCE_PROMPT)
    map_reduce_gpt4_res.append(result)

In [135]:
usage.total_tokens()

29069

#### GPT-4

In [138]:
for i in range(len(map_reduce_gpt4_res)):
    print(f"\n\nCluster: {i}:")
    print(map_reduce_gpt4_res[i])



Cluster: 0:
(1) Main Question Themes:
- Data Science Strategy and Management
- Data Analysis and Insights
- Data Engineering and Infrastructure
- Collaboration and Cross-Functional Work
- Data Governance and Compliance

(2) Representative Example Questions:
- Data Science Strategy and Management: 
  1. "How do you define and execute a data science strategy that ensures our organization's success?"
  2. "What are the best practices for managing a data science team?"
- Data Analysis and Insights: 
  1. "How do you analyze product data to identify problems and ideate product features and process enhancements?"
  2. "What methods do you use to provide insights from data for sound business decisions?"
- Data Engineering and Infrastructure: 
  1. "How do you manage and scale data pipelines from internal and external data sources to support new product launches and drive data quality across data products?"
  2. "What strategies do you use to ensure data quality and security in data analytic

In [140]:
usage = Usage()

In [141]:
map_reduce_gpt35_res = []
for i in range(len(docs)):
    result = run_map_reduce(docs[i], "gpt-3.5-turbo", MAP_PROMPT, REDUCE_PROMPT)
    map_reduce_gpt35_res.append(result)

In [142]:
usage.total_tokens()

27260

### GPT-3.5-turbo

In [143]:
for i in range(len(map_reduce_gpt35_res)):
    print(f"\n\nCluster: {i}:")
    print(map_reduce_gpt35_res[i])



Cluster: 0:
(1) Main Question Themes:
- Data Science Strategy and Leadership
- Data Management and Quality
- Collaboration and Teamwork
- Data Analysis and Insights
- Data Infrastructure and Tools

(2) Representative Example Questions:
- Data Science Strategy and Leadership:
  - "How can we define and execute a data science and analytics roadmap?"
  - "What are the best practices for building a data-driven culture within an organization?"

- Data Management and Quality:
  - "How can we ensure data governance, data security, and data quality?"
  - "What are the key steps in data cleaning and preprocessing to improve data quality?"

- Collaboration and Teamwork:
  - "How can we collaborate with other teams to turn data into critical information and knowledge?"
  - "What are effective strategies for fostering cross-functional collaboration in data science projects?"

- Data Analysis and Insights:
  - "How can we analyze product data to identify problems and ideate product features?"
  -

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
input_doc = '\n\n'.join(filtered_unique_messages_first)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=4000,chunk_overlap=0,separator="\n\n")
# Sanity check
len(text_splitter.split_text(input_doc))

### Targeted Topic Test

In [147]:
reduce_template_string = template = """The following is a list of summaries:
{question_summaries}

Take these and distill it into a final, consolidated list with: 
(1) the top 10 texts related to AI and ML applications.
(2) estimate the proportion of each question

Helpful Answer:"""
REDUCE_PROMPT = PromptTemplate(input_variables=["question_summaries"], template=reduce_template_string)

In [150]:
len(docs[3])

20018

#### GPT-4

In [148]:
result = run_map_reduce(docs[3], "gpt-4", MAP_PROMPT, REDUCE_PROMPT)

In [149]:
print(result)

(1) Top 10 Texts Related to AI and ML Applications:
   - "Experience in developing and applying machine learning models"
   - "Use of data science and analytics to inform strategic decisions"
   - "Proficiency in AI tools and frameworks"
   - "Deployment and scaling of machine learning models in a production environment"
   - "Level of experience and expertise in machine learning and data science"
   - "Knowledge in specific areas of machine learning such as reinforcement learning, deep learning, and natural language processing"
   - "Proficiency in specific tools or languages such as Python, TensorFlow, and PyTorch"
   - "Application of machine learning in specific industries such as insurance, payments, and ad recommendation"
   - "Understanding of the theoretical foundations of machine learning and data science"
   - "Ability to communicate complex data science concepts to non-technical stakeholders"

(2) Estimated Proportions of Each Question:
   - "Experience in developing and app

### GPT3.5 Turbo

In [151]:
usage = Usage()

In [152]:
result = run_map_reduce(docs[3], "gpt-3.5-turbo", MAP_PROMPT, REDUCE_PROMPT)

In [153]:
usage.total_tokens()

4700

In [154]:
print(result)

(1) Top 10 Texts Related to AI and ML Applications:
1) Strategic Communications and Machine Learning: Applying AI to Improve Communication Strategies
2) Ad Recommendation Systems: Machine Learning Approaches for Personalized Advertising
3) Natural Language Processing: Techniques and Applications in AI and ML
4) Computer Vision: Deep Learning Methods for Image Analysis and Recognition
5) Image Processing: Advanced Techniques in AI and ML for Image Enhancement and Analysis
6) AI in Healthcare: Applications of Machine Learning for Medical Diagnosis and Treatment
7) Financial Forecasting: Predictive Analytics and Machine Learning in Finance
8) Autonomous Vehicles: Machine Learning Algorithms for Self-Driving Cars
9) Fraud Detection: AI and ML Techniques for Identifying and Preventing Fraudulent Activities
10) Recommender Systems: Machine Learning Approaches for Personalized Recommendations

(2) Estimated Proportion of Each Question:
a) Machine Learning Expertise: 30%
b) Data Science and An